In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
!pip install keras-tuner

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"], inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()
print(application_type_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Determine which values to replace if counts are less than ...?
replace_application = list(application_type_counts[application_type_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_class_counts = application_df["CLASSIFICATION"].value_counts()
print(application_class_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [7]:
# Determine which values to replace if counts are less than ..?
replace_class = list(application_class_counts[application_class_counts < 1500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [8]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [9]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [10]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

C:\Users\Owner\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:

# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 7
hidden_nodes_layer3 = 3

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                440       
                                                                 
 dense_1 (Dense)             (None, 7)                 77        
                                                                 
 dense_2 (Dense)             (None, 3)                 24        
                                                                 
 dense_3 (Dense)             (None, 1)                 4         
                                                                 
Total params: 545
Trainable params: 545
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints2/",exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:02d}.hdf5"

In [16]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
780/804 [============================>.] - ETA: 0s - loss: 0.6409 - accuracy: 0.6206
Epoch 1: saving model to checkpoints2\weights.01.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.6400 - accuracy: 0.6218
Epoch 2/100
784/804 [============================>.] - ETA: 0s - loss: 0.5749 - accuracy: 0.7099
Epoch 2: saving model to checkpoints2\weights.02.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5743 - accuracy: 0.7098
Epoch 3/100
774/804 [===========================>..] - ETA: 0s - loss: 0.5620 - accuracy: 0.7162
Epoch 3: saving model to checkpoints2\weights.03.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5623 - accuracy: 0.7161
Epoch 4/100
775/804 [===========================>..] - ETA: 0s - loss: 0.5599 - accuracy: 0.7261
Epoch 4: saving model to checkpoints2\weights.04.hdf5
804/804 [==============================] - 1s 2ms/step - loss: 0.5595 - accuracy: 0.7268
Epoch 5/100
789/804 [===================

Epoch 35/100
776/804 [===========================>..] - ETA: 0s - loss: 0.5474 - accuracy: 0.7344
Epoch 35: saving model to checkpoints2\weights.35.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5477 - accuracy: 0.7343
Epoch 36/100
772/804 [===========================>..] - ETA: 0s - loss: 0.5470 - accuracy: 0.7364
Epoch 36: saving model to checkpoints2\weights.36.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7356
Epoch 37/100
789/804 [============================>.] - ETA: 0s - loss: 0.5482 - accuracy: 0.7345
Epoch 37: saving model to checkpoints2\weights.37.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5478 - accuracy: 0.7347
Epoch 38/100
790/804 [============================>.] - ETA: 0s - loss: 0.5472 - accuracy: 0.7352
Epoch 38: saving model to checkpoints2\weights.38.hdf5
804/804 [==============================] - 1s 922us/step - loss: 0.5474 - accuracy: 0.7348
Epoch 39/100
781/804 [========

804/804 [==============================] - 1s 978us/step - loss: 0.5457 - accuracy: 0.7357
Epoch 69/100
804/804 [==============================] - ETA: 0s - loss: 0.5461 - accuracy: 0.7350
Epoch 69: saving model to checkpoints2\weights.69.hdf5
804/804 [==============================] - 1s 966us/step - loss: 0.5461 - accuracy: 0.7350
Epoch 70/100
760/804 [===========================>..] - ETA: 0s - loss: 0.5450 - accuracy: 0.7374
Epoch 70: saving model to checkpoints2\weights.70.hdf5
804/804 [==============================] - 1s 949us/step - loss: 0.5460 - accuracy: 0.7367
Epoch 71/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5459 - accuracy: 0.7355
Epoch 71: saving model to checkpoints2\weights.71.hdf5
804/804 [==============================] - 1s 954us/step - loss: 0.5463 - accuracy: 0.7350
Epoch 72/100
759/804 [===========================>..] - ETA: 0s - loss: 0.5463 - accuracy: 0.7353
Epoch 72: saving model to checkpoints2\weights.72.hdf5
804/804 [===============

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7251 - 328ms/epoch - 1ms/step
Loss: 0.5548385381698608, Accuracy: 0.7251312136650085


In [18]:
# Export our model to HDF5 file
nn2.save("trained_application2.h5")

In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 7
# hidden_nodes_layer3 = 3

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                440       
                                                                 
 dense_5 (Dense)             (None, 7)                 77        
                                                                 
 dense_6 (Dense)             (None, 1)                 8         
                                                                 
Total params: 525
Trainable params: 525
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints3/",exist_ok=True)
checkpoint_path = "checkpoints3/weights.{epoch:02d}.hdf5"

In [22]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200,callbacks=[cp_callback])

Epoch 1/200
787/804 [============================>.] - ETA: 0s - loss: 0.6254 - accuracy: 0.6854
Epoch 1: saving model to checkpoints3\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.6244 - accuracy: 0.6864
Epoch 2/200
789/804 [============================>.] - ETA: 0s - loss: 0.5815 - accuracy: 0.7275
Epoch 2: saving model to checkpoints3\weights.02.hdf5
804/804 [==============================] - 1s 984us/step - loss: 0.5816 - accuracy: 0.7274
Epoch 3/200
796/804 [============================>.] - ETA: 0s - loss: 0.5731 - accuracy: 0.7291
Epoch 3: saving model to checkpoints3\weights.03.hdf5
804/804 [==============================] - 1s 975us/step - loss: 0.5731 - accuracy: 0.7289
Epoch 4/200
796/804 [============================>.] - ETA: 0s - loss: 0.5673 - accuracy: 0.7306
Epoch 4: saving model to checkpoints3\weights.04.hdf5
804/804 [==============================] - 1s 976us/step - loss: 0.5677 - accuracy: 0.7304
Epoch 5/200
785/804 [=============

Epoch 35/200
757/804 [===========================>..] - ETA: 0s - loss: 0.5479 - accuracy: 0.7349
Epoch 35: saving model to checkpoints3\weights.35.hdf5
804/804 [==============================] - 1s 955us/step - loss: 0.5479 - accuracy: 0.7347
Epoch 36/200
756/804 [===========================>..] - ETA: 0s - loss: 0.5473 - accuracy: 0.7337
Epoch 36: saving model to checkpoints3\weights.36.hdf5
804/804 [==============================] - 1s 958us/step - loss: 0.5478 - accuracy: 0.7332
Epoch 37/200
768/804 [===========================>..] - ETA: 0s - loss: 0.5476 - accuracy: 0.7355
Epoch 37: saving model to checkpoints3\weights.37.hdf5
804/804 [==============================] - 1s 947us/step - loss: 0.5476 - accuracy: 0.7354
Epoch 38/200
798/804 [============================>.] - ETA: 0s - loss: 0.5475 - accuracy: 0.7346
Epoch 38: saving model to checkpoints3\weights.38.hdf5
804/804 [==============================] - 1s 975us/step - loss: 0.5476 - accuracy: 0.7346
Epoch 39/200
794/804 [==

Epoch 69/200
756/804 [===========================>..] - ETA: 0s - loss: 0.5429 - accuracy: 0.7364
Epoch 69: saving model to checkpoints3\weights.69.hdf5
804/804 [==============================] - 1s 961us/step - loss: 0.5441 - accuracy: 0.7355
Epoch 70/200
767/804 [===========================>..] - ETA: 0s - loss: 0.5445 - accuracy: 0.7368
Epoch 70: saving model to checkpoints3\weights.70.hdf5
804/804 [==============================] - 1s 943us/step - loss: 0.5442 - accuracy: 0.7364
Epoch 71/200
762/804 [===========================>..] - ETA: 0s - loss: 0.5430 - accuracy: 0.7371
Epoch 71: saving model to checkpoints3\weights.71.hdf5
804/804 [==============================] - 1s 952us/step - loss: 0.5440 - accuracy: 0.7363
Epoch 72/200
778/804 [============================>.] - ETA: 0s - loss: 0.5445 - accuracy: 0.7355
Epoch 72: saving model to checkpoints3\weights.72.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5439 - accuracy: 0.7359
Epoch 73/200
772/804 [====

Epoch 103/200
784/804 [============================>.] - ETA: 0s - loss: 0.5427 - accuracy: 0.7366
Epoch 103: saving model to checkpoints3\weights.103.hdf5
804/804 [==============================] - 1s 992us/step - loss: 0.5430 - accuracy: 0.7366
Epoch 104/200
759/804 [===========================>..] - ETA: 0s - loss: 0.5418 - accuracy: 0.7374
Epoch 104: saving model to checkpoints3\weights.104.hdf5
804/804 [==============================] - 1s 954us/step - loss: 0.5431 - accuracy: 0.7364
Epoch 105/200
754/804 [===========================>..] - ETA: 0s - loss: 0.5439 - accuracy: 0.7357
Epoch 105: saving model to checkpoints3\weights.105.hdf5
804/804 [==============================] - 1s 961us/step - loss: 0.5433 - accuracy: 0.7364
Epoch 106/200
762/804 [===========================>..] - ETA: 0s - loss: 0.5417 - accuracy: 0.7380
Epoch 106: saving model to checkpoints3\weights.106.hdf5
804/804 [==============================] - 1s 945us/step - loss: 0.5429 - accuracy: 0.7369
Epoch 107/20

788/804 [============================>.] - ETA: 0s - loss: 0.5421 - accuracy: 0.7370
Epoch 136: saving model to checkpoints3\weights.136.hdf5
804/804 [==============================] - 1s 984us/step - loss: 0.5427 - accuracy: 0.7367
Epoch 137/200
753/804 [===========================>..] - ETA: 0s - loss: 0.5413 - accuracy: 0.7374
Epoch 137: saving model to checkpoints3\weights.137.hdf5
804/804 [==============================] - 1s 959us/step - loss: 0.5427 - accuracy: 0.7366
Epoch 138/200
768/804 [===========================>..] - ETA: 0s - loss: 0.5413 - accuracy: 0.7377
Epoch 138: saving model to checkpoints3\weights.138.hdf5
804/804 [==============================] - 1s 945us/step - loss: 0.5426 - accuracy: 0.7363
Epoch 139/200
801/804 [============================>.] - ETA: 0s - loss: 0.5426 - accuracy: 0.7366
Epoch 139: saving model to checkpoints3\weights.139.hdf5
804/804 [==============================] - 1s 969us/step - loss: 0.5427 - accuracy: 0.7365
Epoch 140/200
760/804 [===

799/804 [============================>.] - ETA: 0s - loss: 0.5420 - accuracy: 0.7365
Epoch 169: saving model to checkpoints3\weights.169.hdf5
804/804 [==============================] - 1s 972us/step - loss: 0.5421 - accuracy: 0.7364
Epoch 170/200
753/804 [===========================>..] - ETA: 0s - loss: 0.5415 - accuracy: 0.7368
Epoch 170: saving model to checkpoints3\weights.170.hdf5
804/804 [==============================] - 1s 961us/step - loss: 0.5421 - accuracy: 0.7362
Epoch 171/200
750/804 [==========================>...] - ETA: 0s - loss: 0.5416 - accuracy: 0.7365
Epoch 171: saving model to checkpoints3\weights.171.hdf5
804/804 [==============================] - 1s 963us/step - loss: 0.5421 - accuracy: 0.7359
Epoch 172/200
760/804 [===========================>..] - ETA: 0s - loss: 0.5410 - accuracy: 0.7389
Epoch 172: saving model to checkpoints3\weights.172.hdf5
804/804 [==============================] - 1s 953us/step - loss: 0.5423 - accuracy: 0.7373
Epoch 173/200
765/804 [===

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5542 - accuracy: 0.7247 - 299ms/epoch - 1ms/step
Loss: 0.5541885495185852, Accuracy: 0.7246647477149963


In [24]:
# Export our model to HDF5 file
nn3.save("trained_application3.h5")

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 7
# hidden_nodes_layer3 = 3

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Third hidden layer
# nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))


# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 10)                440       
                                                                 
 dense_8 (Dense)             (None, 7)                 77        
                                                                 
 dense_9 (Dense)             (None, 1)                 8         
                                                                 
Total params: 525
Trainable params: 525
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints4/",exist_ok=True)
checkpoint_path = "checkpoints4/weights.{epoch:02d}.hdf5"

In [28]:
# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

Epoch 1/100
772/804 [===========================>..] - ETA: 0s - loss: 0.5986 - accuracy: 0.6966
Epoch 1: saving model to checkpoints4\weights.01.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5975 - accuracy: 0.6979
Epoch 2/100
757/804 [===========================>..] - ETA: 0s - loss: 0.5657 - accuracy: 0.7280
Epoch 2: saving model to checkpoints4\weights.02.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5654 - accuracy: 0.7279
Epoch 3/100
768/804 [===========================>..] - ETA: 0s - loss: 0.5601 - accuracy: 0.7289
Epoch 3: saving model to checkpoints4\weights.03.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5597 - accuracy: 0.7294
Epoch 4/100
804/804 [==============================] - ETA: 0s - loss: 0.5572 - accuracy: 0.7307
Epoch 4: saving model to checkpoints4\weights.04.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5572 - accuracy: 0.7307
Epoch 5/100
764/804 [===================

Epoch 35/100
792/804 [============================>.] - ETA: 0s - loss: 0.5449 - accuracy: 0.7356
Epoch 35: saving model to checkpoints4\weights.35.hdf5
804/804 [==============================] - 1s 981us/step - loss: 0.5450 - accuracy: 0.7356
Epoch 36/100
787/804 [============================>.] - ETA: 0s - loss: 0.5446 - accuracy: 0.7361
Epoch 36: saving model to checkpoints4\weights.36.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5447 - accuracy: 0.7359
Epoch 37/100
764/804 [===========================>..] - ETA: 0s - loss: 0.5443 - accuracy: 0.7359
Epoch 37: saving model to checkpoints4\weights.37.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5446 - accuracy: 0.7357
Epoch 38/100
770/804 [===========================>..] - ETA: 0s - loss: 0.5448 - accuracy: 0.7360
Epoch 38: saving model to checkpoints4\weights.38.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5446 - accuracy: 0.7361
Epoch 39/100
780/804 [========

Epoch 69/100
780/804 [============================>.] - ETA: 0s - loss: 0.5432 - accuracy: 0.7364
Epoch 69: saving model to checkpoints4\weights.69.hdf5
804/804 [==============================] - 1s 929us/step - loss: 0.5429 - accuracy: 0.7365
Epoch 70/100
768/804 [===========================>..] - ETA: 0s - loss: 0.5428 - accuracy: 0.7374
Epoch 70: saving model to checkpoints4\weights.70.hdf5
804/804 [==============================] - 1s 945us/step - loss: 0.5429 - accuracy: 0.7366
Epoch 71/100
791/804 [============================>.] - ETA: 0s - loss: 0.5431 - accuracy: 0.7365
Epoch 71: saving model to checkpoints4\weights.71.hdf5
804/804 [==============================] - 1s 912us/step - loss: 0.5427 - accuracy: 0.7366
Epoch 72/100
792/804 [============================>.] - ETA: 0s - loss: 0.5427 - accuracy: 0.7361
Epoch 72: saving model to checkpoints4\weights.72.hdf5
804/804 [==============================] - 1s 975us/step - loss: 0.5426 - accuracy: 0.7364
Epoch 73/100
779/804 [==

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5530 - accuracy: 0.7247 - 292ms/epoch - 1ms/step
Loss: 0.5529974102973938, Accuracy: 0.7246647477149963


In [30]:
# Export our model to HDF5 file
nn4.save("trained_application4.h5")